In [6]:
import json
from pathlib import Path

from scipy.interpolate import RBFInterpolator
from scipy.ndimage import rotate
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import numpy as np
import vedo

from histalign.backend.ccf.paths import get_structure_mask_path
from histalign.backend.io import load_image
from histalign.backend.models import AlignmentSettings, Resolution
from histalign.backend.registration import Registrator
from histalign.backend.workspace import VolumeSlicer

vedo.settings.default_backend = "vtk"

In [23]:
def process_alignment(path: str, structure_name: str, master_array: np.ndarray) -> None:
    settings = get_settings(path)

    image = load_image(settings.histology_path)
    masked_image = mask_off(image, structure_name, settings)

    insert_into_array(image, master_array, settings)


def get_settings(path: str) -> AlignmentSettings:
    with open(path) as handle:
        return AlignmentSettings(**json.load(handle))


def mask_off(
    image: np.ndarray, structure_name: str, alignment_settings: AlignmentSettings
) -> np.ndarray:
    mask_volume = VolumeSlicer(
        path=get_structure_mask_path(
            structure_name, alignment_settings.volume_settings.resolution
        ),
        resolution=alignment_settings.volume_settings.resolution,
    )

    mask_slice = mask_volume.slice(alignment_settings.volume_settings)

    registrator = Registrator(False, True)
    registered_image = registrator.get_forwarded_image(alignment_settings, image)

    return np.where(mask_slice > 0, registered_image, 0)


def insert_into_array(
    image: np.ndarray, target_array: np.ndarray, settings: AlignmentSettings
) -> np.ndarray:
    slicer = VolumeSlicer(
        path=get_structure_mask_path("root", Resolution.MICRONS_100),
        resolution=Resolution.MICRONS_100,
        lazy=False,
    )
    volume = slicer.volume._volume
    array = volume.tonumpy()

    # array = rotate(array, 45)
    # volume2 = vedo.Volume(array)

    if settings.volume_settings.orientation != "sagittal":
        raise ValueError("Panic: not implemented")

    volume2 = volume.clone()

    settings.volume_settings.pitch = 20
    settings.volume_settings.yaw = 20

    print(f"{settings.volume_settings.pitch=}")
    print(f"{settings.volume_settings.yaw=}")

    volume2 = volume2.rotate_x(-settings.volume_settings.pitch, around=volume2.center())
    volume2 = volume2.rotate_y(settings.volume_settings.yaw, around=volume2.center())

    volume3 = vedo.Volume(volume2.tonumpy()[..., : volume2.shape[2] // 2])

    vedo.Plotter().show([volume2, volume3], axes=3)


def imshow(image: np.ndarray) -> None:
    _, axes = plt.subplots()

    axes.imshow(image)
    axes.axis(False)

    plt.show()

In [8]:
# alignment_directory = Path("/home/ediun/git/histalign/projects/project_310724_full/7d9af388b0")
alignment_directory = Path(
    "/home/ediun/git/histalign/projects/project_temp4/12d4fcac65"
)
# alignment_directory = Path("/home/ediun/git/histalign/projects/project_temp4/ed8140f0dd")

In [9]:
master_array = np.zeros(shape=(528, 320, 456))

In [24]:
i = 0
target = 0

for file in alignment_directory.iterdir():
    if i < target:
        i += 1
        continue

    if file.suffix != ".json" or file.stem == "metadata":
        continue

    process_alignment(str(file), "root", master_array)

    break

settings.volume_settings.pitch=20
settings.volume_settings.yaw=20
